<a href="https://colab.research.google.com/github/Cloud-Course-Group-Phoenix/Project-Pheonix/blob/Dev/Logic/SensorDataProcessor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
%pip install -q importnb
%pip install -q paho-mqtt

try:
    #Clone the GitHub repository if not already present
    if not os.path.exists("/content/Project-Pheonix"):
        !git clone https://github.com/Cloud-Course-Group-Phoenix/Project-Pheonix.git /content/Project-Pheonix

    # Change directory to project root
    %cd /content/Project-Pheonix

    # Checkout the 'dev' branch
    !git fetch origin -q
    !git checkout Dev -q

    # Add project directory to Python path
    sys.path.append("/content/Project-Pheonix/Logic")
    from importnb import Notebook
    with Notebook():
        import CloudDB as dbService
    import time
    import json
    import paho.mqtt.client as mqtt
    from datetime import datetime
except Exception as e:
    print("❌ Setup failed:", str(e))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 4.6 MB/s eta 0:00:00
Cloning into '/content/Project-Pheonix'...
remote: Enumerating objects: 519, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 519 (delta 57), reused 13 (delta 13), pack-reused 425 (from 1)
Receiving objects: 100% (519/519), 1.89 MiB | 13.64 MiB/s, done.
Resolving deltas: 100% (284/284), done.
/content/Project-Pheonix


In [ ]:
# Mqtt data processor microservice
class MqttConnection:
    # Class variable to store the singleton instance

    def __init__(self):
        # Make sure the instance is only created once
          self.client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
          self.connected = False
          self.last_message_time = time.time()  # Track last message received
          self.timeout_seconds = 10  # 10 second timeout
          self.should_stop = False  # Flag to control main loop


    # Connection Handler
    def on_connect(self, client, userdata, flags, rc, properties=None):
        if rc == 0:
            self.connected = True
            print("Connected to MQTT Broker!\nSubscribing to topics")

            # Subscribe to the relevant topics
            client.subscribe("braude/D106/indoor")
            client.subscribe("braude/D106/outdoor")

            print("Successfully subscribed to topics!")
            # Reset the last message time when connected
            self.last_message_time = time.time()
        else:
            print(f"Failed to connect, return code {rc}")
            self.connected = False


    # Disconnection Handler
    def on_disconnect(self, client, userdata, flags, rc, properties=None):
        self.connected = False
        if rc != 0:
            # Non-zero return code means unexpected disconnection
            for i in range(5):  # Try 5 times with increasing backoff
                wait_time = (i + 1) * 2  # Increasing backoff (2s, 4s, 6s, 8s, 10s)
                print(f"Unexpected disconnection. Attempting to reconnect (try {i+1}/5) in {wait_time}s...")
                time.sleep(wait_time)
                try:
                    client.reconnect()
                    print("Reconnection attempted")
                    break
                except Exception as e:
                    print(f"Reconnection attempt failed: {e}")

    # Receiver Handler
    def on_message(self, client, userdata, msg):
        # Update last message time when a message is received
        self.last_message_time = time.time()

        topic = msg.topic
        payload = msg.payload.decode('utf-8')  # Decode the byte string to a string
        print(f"Message received on topic {topic} at {datetime.now().strftime('%H:%M:%S')}")

        try:
            sensor_data = json.loads(payload)
            if topic == "braude/D106/indoor":
                dbService.insert_to_db_sensor(f"indoor/{int(time.time())}", sensor_data)
            elif topic == "braude/D106/outdoor":
                dbService.insert_to_db_sensor(f"outdoor/{int(time.time())}", sensor_data)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            print(f"Problematic payload: {payload}")

    def check_timeout(self):
        """Check if timeout has been reached"""
        current_time = time.time()
        time_since_last_message = current_time - self.last_message_time

        if time_since_last_message >= self.timeout_seconds:
            print(f"\n⏰ No messages received for {self.timeout_seconds} seconds. Stopping MQTT client...")
            return True
        return False

    def stop_client(self):
        """Stop the MQTT client gracefully"""
        print("Stopping MQTT client...")
        self.should_stop = True
        self.client.loop_stop()
        self.client.disconnect()
        self.connected = False
        print("MQTT client stopped.")

    def mqtt_handler(self):
        # Register callbacks
        self.client.on_connect = self.on_connect
        self.client.on_disconnect = self.on_disconnect
        self.client.on_message = self.on_message

        try:
            print("Attempting to connect to MQTT broker...")
            conn_result = self.client.connect("test.mosquitto.org", 1883, keepalive=60)
            print(f"Connection attempt result code: {conn_result}")

            # Start the loop in the background
            self.client.loop_start()

            # Wait for connection to establish
            timeout = time.time() + 10  # 10 second timeout
            while time.time() < timeout and not self.connected:
                time.sleep(0.2)

            if self.connected:
                return True
            return False

        except Exception as e:
            print(f"Connection failed with error: {e}")
            return False

    def run_with_timeout(self):
        """Run the MQTT client with timeout monitoring"""
        print(f"Starting MQTT client with {self.timeout_seconds}-second timeout...")
        print("Waiting for messages...")

        try:
            while self.connected and not self.should_stop:
                # Check for timeout every second
                time.sleep(1)

                if self.check_timeout():
                    self.stop_client()
                    break

                # Show a dot every 5 seconds to indicate the client is still running
                current_time = time.time()
                if int(current_time) % 5 == 0:
                    time_waiting = int(current_time - self.last_message_time)


        except KeyboardInterrupt:
            print("\n⚠️ Interrupted by user")
            self.stop_client()
        except Exception as e:
            print(f"\n❌ Error in main loop: {e}")
            self.stop_client()


# Initialize the global MQTT connection as a singleton
print("Initializing MQTT connection...")
mqtt_connection = MqttConnection()
# Start the MQTT handler if it's a new instance
if not mqtt_connection.connected:
    success = mqtt_connection.mqtt_handler()
    print(f"MQTT connection result: {'Connected' if success else 'Failed'}")

if mqtt_connection.connected:
    print("\n🚀 Starting MQTT client with 10-second timeout...")
    time.sleep(2)  # Brief pause before starting
    mqtt_connection.run_with_timeout()
else:
    print("❌ Failed to connect to MQTT broker")


Initializing MQTT connection...
Attempting to connect to MQTT broker...
Connection attempt result code: 0
Connected to MQTT Broker!
Subscribing to topics
Successfully subscribed to topics!
MQTT connection result: Connected

🚀 Starting MQTT client with 10-second timeout...
Starting MQTT client with 10-second timeout...
Waiting for messages...

⏰ No messages received for 10 seconds. Stopping MQTT client...
Stopping MQTT client...
MQTT client stopped.
